# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7f9ab017a400>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f9ab01c9a20>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |



NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7f9ab017a400>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f9ab01c9a20>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.48199999999999998

In [11]:
net.fit(X, y, num_epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       0 |      0.20988 | 0.04648 |
|       1 |      0.09319 | 0.03878 |
|       2 |      0.07161 | 0.03858 |
|       3 |      0.06325 | 0.03815 |
|       4 |      0.05884 | 0.03690 |
|       5 |      0.05591 | 0.03786 |
|       6 |      0.05368 | 0.03741 |
|       7 |      0.05181 | 0.03900 |
|       8 |      0.05018 | 0.03766 |
|       9 |      0.04868 | 0.04036 |
|      10 |      0.04731 | 0.04240 |
|      11 |      0.04602 | 0.04241 |
|      12 |      0.04480 | 0.04307 |
|      13 |      0.04366 | 0.03870 |
|      14 |      0.04251 | 0.03771 |
|      15 |      0.04154 | 0.03590 |
|      16 |      0.04051 | 0.03740 |
|      17 |      0.03958 | 0.03648 |
|      18 |      0.03861 | 0.03590 |
|      19 |      0.03

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7f9ab017a400>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f9ab01c9a20>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba = net.predict_proba(X)

In [13]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.99280000000000002

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [14]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [15]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [16]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [17]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [18]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.954409 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.950210 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.960384 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.975405 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.983803 -   0.5s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   18.7s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=Fa...ject at 0x7f9ab01c9a20>,),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_epochs': [5, 10], 'update__learning_rate': [0.1, 0.5], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [19]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 100, 'max_epochs': 10, 'update__learning_rate': 0.5},
 0.98319999999999996)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [20]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [21]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [22]:
net = NeuralNetRegressor(l2, verbose=1)

In [23]:
net.set_params(update__learning_rate=0.0001)

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7f9ab017a6a0>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f9ab017a668>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

### Train

In [24]:
net.fit(X, y, num_epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |    100 |
|   1 | dense  |    200 |
|   2 | dense  |      1 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       0 |  25477.56836 | 0.06897 |
|       1 |  24711.97656 | 0.03186 |
|       2 |  22082.50391 | 0.03216 |
|       3 |  14621.14258 | 0.03380 |
|       4 |   4874.32520 | 0.03194 |
|       5 |    848.73694 | 0.02977 |
|       6 |    290.25079 | 0.02963 |
|       7 |    225.20493 | 0.03236 |
|       8 |    203.59399 | 0.03186 |
|       9 |    189.39383 | 0.03389 |
|      10 |    178.23537 | 0.03279 |
|      11 |    168.81763 | 0.03295 |
|      12 |    160.55678 | 0.03320 |
|      13 |    153.11763 | 0.03267 |
|      14 |    146.26237 | 0.03077 |
|      15 |    139.88733 | 0.03060 |
|      16 |    133.89516 | 0.03199 |
|      17 |    128.23994 | 0.03418 |
|      18 |    122.87547 | 0.03229 |
|      19 |    117.7

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7f9ab017a6a0>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7f9ab017a668>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

## Saving and restoring

### Save previous net

In [25]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

115.248460207


In [26]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [27]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [28]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

115.248460207
